In [1]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
import statistics
import time

In [2]:
#視 SampleSize, bias 產生data
def GenerateData(SampleSize, bias):
    
    #利用random.uniform產生data
    data = np.random.uniform(-1,1,100000)

    #以 bias為 bionomial的 p值，決定對應某x的y是否變號
    data_y = []
    noise = np.random.binomial(1, bias, 100000)
    for d in data:
        data_y.append([d,np.sign(d)*(-1)])
    for i in range(100000):
        if noise[i]==1:
            data_y[i][1]=data_y[i][1]*(-1)
    
    #分割 train set, test set
    train, test = train_test_split(data_y, test_size=(1-SampleSize/100000), \
                                   random_state=random.randint(0, 10000))
    train.sort()
    
    return train, test

In [4]:
def FindG(sample):

    #將各對 theta, s加入hypothesis list
    theta,s = -1, -1

    H = []
    H.append([-1,-1])
    H.append([-1,1])
    
    for i in range(len(sample)-1):
        t = (sample[i][0] + sample[i+1][0]) / 2
        H.append([t, -1])
        H.append([t, 1])
    
    Ein = 1
    
    #測試hypothesis list中的各對 theta, s
    #取滿足ein最小且同ein時 theta+s最小的組合
    
    for h in H:
        error_h = Error(sample, h[0], h[1])
        if error_h <= Ein :
            Ein = error_h
            theta = h[0]
            s = h[1]
        if (error_h == Ein) and (( h[0]+ h[1]) <(theta + s)):
            Ein = error_h
            theta = h[0]
            s = h[1]

    #回傳 Ein, theta, s
    return Ein, theta, s

In [5]:
#計算ein/eout用，即計算 y!= h_theta, s(x)的比例
def Error(sample, theta, s):
    
    error = 0
    for d in sample:
        if d[1] != (s * np.sign(d[0] - theta)):
            error+=1
            
    return error/len(sample)

In [6]:
def Eout_Ein(SampleSize, Bias):
    
    #生成 train, test set
    train, test = GenerateData(SampleSize, Bias)
    
    #找到符合ein最小的function G，回傳 G 的Ein, theta, s
    Ein, theta, s = FindG(train)
    
    #代入 G ，計算 Eout
    Eout = Error(test, theta, s)
    
    #回傳Eout - Ein
    return (Eout - Ein)

In [7]:
#重複實驗100000次並計算mean
def mean(SampleSize, Bias):
    diff = []
    for i in range(10000):
        diff.append(Eout_Ein(SampleSize, Bias))
    return statistics.mean(diff)

In [ ]:
#因為要跑太久這次就不重新執行了

#2-16
print(mean(2,0))

#之前跑的結果0.2956868137362747
#選 d 0.3

In [ ]:
#2-17
print(mean(20,0))

#之前跑的結果0.024131007201440288
#選 b 0.02

In [ ]:
#2-18
print(mean(2,0.1))

#之前跑的結果0.37656463129262585
#選 e 0.4

In [ ]:
#2-19
print(mean(20,0.1))

#之前跑的結果0.050938170634126825
#選 c 0.05

In [ ]:
#2-20
print(mean(200,0.1))

#之前跑的結果0.0051518086172344674
#選 a 0.00